In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

dataset = '/content/drive/MyDrive/Colab Notebooks/JSL/point_history.csv'
model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/gesture_classifier.hdf5'

In [ ]:
'''
for dataset_index, X_data in enumerate(X_dataset):
    for data_index in range(0, len(X_data), 2):
        if data_index == 0:
            image_width = X_data[data_index]
            image_height = X_data[data_index + 1]
            continue

        if data_index == 2:
            base_x = X_data[data_index]
            base_y = X_data[data_index + 1]

        X_data[data_index] = (X_data[data_index] - base_x) / image_width
        X_data[data_index + 1] = (X_data[data_index + 1] - base_y) / image_height


X_dataset = np.delete(X_dataset, [0], 1)
'''

In [9]:
# 指文字の数
NUM_CLASSES = 50

TIME_STEPS = 21
DIMENSION = 2

X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)
X_train = X_train.reshape(-1, TIME_STEPS, DIMENSION)
X_test = X_test.reshape(-1, TIME_STEPS, DIMENSION)

In [10]:
activation_='tanh'

model = Sequential()
model.add(LSTM(64, input_shape=(TIME_STEPS, DIMENSION)))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['accuracy']
)

# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
182/186 [============================>.] - ETA: 0s - loss: 1910.0609 - accuracy: 0.0131
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/gesture_classifier.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


186/186 [==============================] - 7s 9ms/step - loss: 1909.1849 - accuracy: 0.0131 - val_loss: 1898.5725 - val_accuracy: 0.0054
Epoch 2/1000
182/186 [============================>.] - ETA: 0s - loss: 1907.9062 - accuracy: 0.0133
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/gesture_classifier.hdf5
186/186 [==============================] - 1s 5ms/step - loss: 1909.1858 - accuracy: 0.0132 - val_loss: 1898.5724 - val_accuracy: 0.0130
Epoch 3/1000
182/186 [============================>.] - ETA: 0s - loss: 1906.5603 - accuracy: 0.0133
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/gesture_classifier.hdf5
186/186 [==============================] - 1s 5ms/step - loss: 1909.1849 - accuracy: 0.0132 - val_loss: 1898.5725 - val_accuracy: 0.0101
Epoch 4/1000
181/186 [============================>.] - ETA: 0s - loss: 1907.1060 - accuracy: 0.0126
Epoch 4: saving model to /content/drive/MyDrive/Co

In [11]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))

# 推論モデルとして保存
model.save(model_save_path, include_optimizer=False)

1/1 [==============================] - 0s 429ms/step


### 量子化

In [12]:
# 推論モデルを読み込み
import tensorflow as tf
from tensorflow import keras

model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/gesture_classifier.hdf5'
tflite_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier/keypoint_classifier.tflite'

model = tf.keras.models.load_model(model_save_path)

In [13]:
# モデルを変換(量子化)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# TensorFlow Liteでサポートされていない操作を含むモデルに対応するための設定
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]

converter._experimental_lower_tensor_list_ops = False
converter.experimental_new_converter = True

In [14]:
tflite_quantized_model = converter.convert()

In [ ]:
open(tflite_save_path, 'wb').write(tflite_quantized_model)

In [16]:
# 推論テスト
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 推論実施の前に、入力データを適切な形状にリシェイプ
input_data = np.array([X_dataset[0]]).reshape(1, TIME_STEPS, DIMENSION)

# 推論実施
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

#print(np.squeeze(tflite_results))
#print(np.argmax(np.squeeze(tflite_results)))

### 入力テンソルの形状確認

In [18]:
# モデルのパス
tflite_model_path = tflite_save_path

# TensorFlow Lite インタープリタの初期化
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# 入力ディテールの取得
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])  # 入力テンソルの形状を表示

[ 1 21  2]
